In [1]:
import numpy as np
import pandas as pd
import re
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import keras
import tensorflow
# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding
from keras.layers import LSTM, Input, TimeDistributed, SimpleRNN
from keras.models import Model
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import Bio.GA.Crossover.TwoPoint

# Import the backend
from keras import backend as K

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

/Users/c/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/c/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/c/anaconda3/lib/python3.6/site-packages/Bio/GA/__init__.py:14: BiopythonDeprecationWarning: Bio.GA has been deprecated, and we intend to remove it in a future release of Biopython. Please consider using DEAP instead.  If you would like to continue using Bio.GA, please contact the Biopython developers via the mailing list or GitHub.
  BiopythonDeprecationWarning)


In [2]:
with open('dna.txt', 'r') as myfile:
  dna = myfile.read()
dna_basepairs = dna
dna_basepairs = re.sub(r'[0-9]([0-9])?([0-9])?([0-9])?([0-9])?([0-9])?', '', dna_basepairs)
dna_basepairs = re.sub(r'\n', '', dna_basepairs)
dna_basepairs = re.sub(r'\s+', '', dna_basepairs).strip()

Create population of arrays that span 50 index values.

In [3]:
def population(data):
    sequence_one = data[:78122]
    sequence_two = data[78123:]
    
    array = np.zeros((len(sequence_one),8))
    vectorize_sequence = {
        'a' : 1,
        't' : 2,
        'g' : 3,
        'c' : 4
    }

    for base in range(len(sequence_one)):
        base_one = sequence_one[base]
        base_two = sequence_two[base]
        array[base, vectorize_sequence[base_one]] = 1
        array[base, vectorize_sequence[base_two] + 3] = 1
        
    similarity_array = np.zeros((len(array),1))
    similarity_array = np.asarray([similarity_array[i] == 1 if np.equal(array[i, :4], array[i, 4:]).all() else similarity_array[i] == 0 for i in range(len(array))])
        
    
    return [np.asarray(array).reshape(1, 78122, 8), similarity_array.reshape(1, 78122, 1)]

In [4]:
sequences = population(dna_basepairs)
comparison = sequences[0].reshape(1, 78122, 8)
similarity_array = sequences[1].reshape(1, 78122, 1)

In [5]:
print(comparison.shape)
print(similarity_array.shape)

(1, 78122, 8)
(1, 78122, 1)


looping through previous and current index of the population in order to pass each sequence through the RNN for evaluation

In [6]:
network_population = []
for i in range(30):
    model = Sequential()
    # add Long short term memory RNN and a dense layer. Compile the model
    model.add(LSTM(10, input_shape=(comparison.shape[1],8), activation='tanh', return_sequences=True, use_bias=True))
    model.add(Dense(1, activation='tanh', use_bias=True))
    network_population.append(model)

In [ ]:
for i in network_population:
    mse_vec = np.zeros(len(network_population))
    
    for i in range(len(network_population)):
        def mse(y_true, y_pred):
            y_true = y_true.reshape(1, y_true.shape[1])
            y_pred = y_pred.reshape(1, y_pred.shape[1])
            mse = mean_squared_error(y_true, y_pred)
            return mse

        y_pred = model.predict(comparison)
        mse = mse(similarity_array, y_pred)
        mse_vec[i] = mse
    print(mse_vec) 
#     sorted_mse_idx = np.argsort(mse_vec)
#     fittest_mse = sorted_mse_idx[:10]
#     weakest_mse = sorted_mse_idx[10:]
    
#     for network in fittest_mse:
#         model.get_weights()

In [ ]:
#train, validate, test = np.split(sequences, [int(.5*len(sequences)), int(.75*len(sequences))])
#view the various lengths of the split data.
#print('Train length:', len(train), '\n', 'Test length:', len(test), '\n', 'Validation length:', len(validate))